In [1]:

import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nana-\anaconda3\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv("../data/tweets_1.csv")


In [3]:
x= df['text'][df['Unnamed: 0']==2]
print(x)

2    The media and establishment want me out of the race so badly -  I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA
Name: text, dtype: object


In [4]:
df.head(10)

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA",DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\r\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz,DonaldTrump
5,5,4h4 hours ago,786950598826532864,/realDonaldTrump/status/786950598826532864,False,WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIGN’S ANTI-CATHOLIC BIGOTRY: \r\nhttp://bit.ly/2dcbtvkCrooked,DonaldTrump
6,6,Oct 13,786589172911964160,/realDonaldTrump/status/786589172911964161,False,Thank you! #MAGA #AmericaFirstpic.twitter.com/fG313wjlKm,DonaldTrump
7,7,Oct 12,786340623804751872,/realDonaldTrump/status/786340623804751872,False,"I will be in Cincinnati, Ohio tomorrow night at 7:30pm- join me! #OhioVotesEarly #VoteTrumpPence16 \r\nTickets: https://www.donaldjtrump.com/schedule/register/cincinnati-oh2/ …pic.twitter.com/XUFuGc4Fg5",DonaldTrump
8,8,Oct 12,786201435486781440,/realDonaldTrump/status/786201435486781440,False,Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system!,DonaldTrump
9,9,Oct 11,786054986534969344,/realDonaldTrump/status/786054986534969344,False,Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out & #VoteTrumpPence16 on 11/8! #MAGApic.twitter.com/e90a2AL7vO,DonaldTrump


In [5]:

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [6]:
df['text'] = df['text'].astype(str)

In [7]:
df['text'] = df['text'].astype(str)
df['cleaned'] = df['text'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)


In [8]:
EMBEDDING_FILE = "../models/GoogleNews-vectors-negative300.bin.gz"
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


In [9]:
#ssplit en plusieurs mots la description cleaned

import time
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [10]:
# Entrainement surr google model
%time
google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)



google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

Wall time: 0 ns


(0, 0)

# LIMIT MODEL TRAINING


In [11]:
google_model.wv.most_similar(positive=["steve"]) #model google des mots reconus on peut tester des trucs ex: ok, hello, etc

[('dave', 0.7480320334434509),
 ('jeff', 0.6910731196403503),
 ('jason', 0.6878510117530823),
 ('robert', 0.6830391883850098),
 ('george', 0.6792417764663696),
 ('kevin', 0.676276683807373),
 ('todd', 0.6758198738098145),
 ('ryan', 0.6735630035400391),
 ('greg', 0.6706033945083618),
 ('michael', 0.6679339408874512)]

In [12]:
# genere average word2dec pour chaque champ
def vectors(x):
    
    
    global word_embeddings
    word_embeddings = []

    
    for line in df['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)
    
    return len(word_embeddings)


In [13]:
def essentiel(df,text):
    df = df.append({'Unnamed: 0': 17216,'date':"dateeclate",'id':1,'link':"/realDonaldTrump/status/1784884848", 'retweet':"False", 'text':text,'author':"DonalTrump"}, ignore_index=True)
    df['text'] = df['text'].astype(str)
    df['cleaned'] = df['text'].apply(_removeNonAscii)
    df['cleaned'] = df.cleaned.apply(func= make_lower_case)
    df['cleaned'] = df.cleaned.apply(func=remove_stop_words)
    df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
    df['cleaned'] = df.cleaned.apply(func=remove_html)
    return (df)
    

In [18]:

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv("../data/tweets_1.csv")

#df = essentiel(df,"Love you")


def recommendations(text):
    df = essentiel(df,text)
    vectors(df)
    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
    books = df[['text','id']]
    indices = pd.Series(df.index, index = df['text'])
    idx = indices[text]  
    sim_scores = list(enumerate(cosine_similarities[idx]))    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:21]
    book_indices = [i[0] for i in sim_scores]
    recommend = books.iloc[book_indices]

    
    return(recommend)

       Unnamed: 0        date                  id  \
0      0           Oct 7       784609194234306560   
1      1           Oct 10      785608815962099712   
2      2           Oct 8       784840992734064640   
3      3           Oct 8       784767399442653184   
4      4           Oct 10      785561269571026944   
...   ..              ...                     ...   
17212  17212       8 May 2009  1741160716           
17213  17213       8 May 2009  1737479987           
17214  17214       4 May 2009  1701461182           
17215  17215       4 May 2009  1698308935           
17216  17216       dateeclate  1                    

                                             link retweet  \
0      /realDonaldTrump/status/784609194234306560  False    
1      /realDonaldTrump/status/785608815962099712  False    
2      /realDonaldTrump/status/784840992734064641  False    
3      /realDonaldTrump/status/784767399442653184  False    
4      /realDonaldTrump/status/785561269571026946  False  

In [16]:
recommendations("I hate you")

KeyError: 'I hate you'